# Introduction to Vcc Glitch Attacks

In [95]:
#Parameters
SCOPETYPE = 'CWNANO'
PLATFORM = 'CWNANO'
sample_size = 5 #number of times to check each glitch location

## Background on Vcc (Power) Glitching

The previous clock glitching tutorials looked into the assumption of a constant clock. But instead we can modify the voltage of the device, causing for example a failure to correctly read a memory location or otherwise cause havoc with the proper functioning.

An example of a successful Vcc glitch is shown in the following figures (Vcc in blue, clock in red):

![](https://wiki.newae.com/images/4/4f/Vccglitch_working.png)

![](https://wiki.newae.com/images/6/60/Vccglitch_working_zoom.png)

Even more so than clock glitching, Vcc glitching is highly sensitive to glitch offset and width. While the above glitch was successful, the following was not:

![](https://wiki.newae.com/images/b/b6/Vccglitch_notworking_zoom.png)

Vcc glitching is also very sensitive to the shape of the glitch: things like board layout and the distance between where the glitch is inserted and the target can make the difference between successful and unsuccessful glitches.

Despite these additional complications, Vcc glitching is an extremely useful tool as it allows attacks on targets that do not run off of external clock inputs.

### Background on Glitch Generation

For more details, please see [Fault_1-Introduction_to_Clock_Glitching](Fault_1-Introduction_to_Clock_Glitch_Attacks.ipynb), this tutorials assumes you have already performed the clock glitching tutorial.

The glitch generation hardware is the same as used in the clock glitching attack. The generated glitches are synchronous to the device clock, and inserted at a precise offset from the clock edge.

Glitches can be inserted continuously or triggered by some event. The following figure shows the generation of two glitches:

![](https://wiki.newae.com/images/9/95/Glitchgen-mux-glitchonly.png)

The VCC glitching method here uses an electronic switch (a MOSFET) to short the power line to GND at specific instances. The following figure shows the basic function of this system:

![](https://wiki.newae.com/images/8/82/Glitch-vccglitcher.png)

This method allows use with the standard side-channel analysis development board, which has resistors inserted into the VCC lines already. The downside of this method is that it can only generate short glitches, since the power consumption through the shunt resistor will short out the resistor.

The MOSFET glitching hardware is built into the ChipWhisperer-Lite (both CW1173 and CW1180) board.

## Setting up Firmware

During this tutorial, we will once again be working off the `glitch-simple` project.

Now navigate to the `glitch-simple` folder and open `glitchsimple.c` in a code editor. Find the `glitch_infinite()` function:

```C
void glitch_infinite(void)
{
    char str[64];
    unsigned int k = 0;
    //Declared volatile to avoid optimizing away loop.
    //This also adds lots of SRAM access
    volatile uint16_t i, j;
    volatile uint32_t cnt;
    while(1){
        cnt = 0;
        trigger_high();
        trigger_low();
        for(i=0; i<200; i++){
            for(j=0; j<200; j++){
                cnt++;
            }
        }
        sprintf(str, "%lu %d %d %d\n", cnt, i, j, k++);
        uart_puts(str);
    }
}
```

As you can see, this function enters into an infinite loop with two inner loops that increment three variables (`cnt`, `i`, and `j`).  These are sent back over serial along with an overall loop counter. During normal operation, we should receive `40000 200 200 $k` (where `$k` is the value of the loop counter `k`). Our objective will be to insert a Vcc glitch such that one or more of the numbers that we get back are incorrect.

Then build the firmware:

In [96]:
%%bash -s "$PLATFORM"
cd ../../hardware/victims/firmware/glitch-simple
make PLATFORM=$1 CRYPTO_TARGET=NONE FUNC_SEL=GLITCH_INF

rm -f -- glitchsimple-CWNANO.hex
rm -f -- glitchsimple-CWNANO.eep
rm -f -- glitchsimple-CWNANO.cof
rm -f -- glitchsimple-CWNANO.elf
rm -f -- glitchsimple-CWNANO.map
rm -f -- glitchsimple-CWNANO.sym
rm -f -- glitchsimple-CWNANO.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- glitchsimple.s simpleserial.s stm32f0_hal_nano.s stm32f0_hal_lowlevel.s
rm -f -- glitchsimple.d simpleserial.d stm32f0_hal_nano.d stm32f0_hal_lowlevel.d
rm -f -- glitchsimple.i simpleserial.i stm32f0_hal_nano.i stm32f0_hal_lowlevel.i
.
-------- begin --------
arm-none-eabi-gcc (GNU Tools for Arm Embedded Processors 7-2018-q2-update) 7.3.1 20180622 (release) [ARM/embedded-7-branch revision 261907]
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: glitchsimple.c
arm-none-eabi-gcc -c -mcpu=cortex-m0 -I. -DGLITCH_INF -mthumb -mfloat-abi=soft -ffunct

## Attack Script

### Setup

Now that we've studied the code and have an objective, we can start building our attack script. We'll start by connecting to and setting up the ChipWhisperer, then programming it. As usual, make sure you modify `fw_path` with the path to the file you built in the last step.

In [97]:
%run "Helper_Scripts/Setup.ipynb"

In [98]:
fw_path = "../../hardware/victims/firmware/glitch-simple/glitchsimple-{}.hex".format(PLATFORM)

In [99]:
cw.programTarget(scope, prog, fw_path)

Detected unknown STM32F ID: 0x444
Extended erase (0x44), this can take ten seconds or more
Attempting to programming 6043 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 6043 bytes


Like with clock glitching, Vcc glitching may crash the target, requiring a reset. Like with the previous tutorial, we'll use `reset_target()` from `Helper_Scripts/Setup.ipynb`.

Now that we have some of the basic setup done, let's make sure the firmware works as we expect. If we reset the target and wait a second, then print the serial data we got back, we should see a number of lines of the form `40000 200 200 $k`.

In [100]:
reset_target(scope)
target.ser.flush()
time.sleep(1)
num_char = target.ser.inWaiting()
resp = target.ser.read(num_char)
print(resp)

40000 200 200 0
40000 200 200 1
40000 200 200 2
40000 200 200 3
40000 200 200 4
40000 200 200 5
40000 200 200 6
40000 200 200 7


### Glitch Setup

First, we'll setup the glitch module itself. Most of these settings should look familiar from the previous tutorial with a few new additions:

* Instead of setting the clock source for the target to be the glitch module, we instead set the low power MOSFET's input to be the glitch module by setting `scope.io.glitch_lp` to `True`. The ChipWhisperer-Lite also has a high power MOSFET, but we won't be using that in this tutorial.
* Instead of setting the glitch output to something like "clock_xor", we instead set it to "glitch_only", since we don't want Vcc of the target to be oscillating with our clock.

For the more specific settings (offset, width, repeat, etc), this will depend on both the target and when you got your CW-Lite: Newer versions of the CW-Lite use a different glitch MOSFET, which changes the settings required for getting a glitch.

In [160]:
from collections import namedtuple
if PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
    scope.glitch.clk_src = "clkgen"
    scope.glitch.output = "glitch_only"
    scope.glitch.trigger_src = "ext_single"
    scope.glitch.width = 35
    scope.glitch.offset = -17.4
    scope.glitch.repeat = 1
    
    Range = namedtuple('Range', ['min', 'max', 'step'])
    width_range = Range(38.5, 39.1, 0.4)
    offset_range = Range(-28.4, -28.125, 0.4)
    scope.glitch.offset_fine = 24
    def glitch_on(scope):
        scope.io.glitch_lp = False
        scope.io.glitch_hp = True
    def glitch_off(scope):
        scope.gio.glitch_hp = False
    glitch_on(scope)
    scope.glitch.ext_offset = 2186
    print(scope.glitch)
elif PLATFORM == "CWNANO" and SCOPETYPE == "CWNANO":
    scope.glitch.ext_offset = 546
    scope.adc.clk_freq = 7.5E6
    scope.glitch.repeat = 6
    def glitch_on(scope):
        pass
    def glitch_off(scope):
        pass
    pass #later

In [161]:
print(scope)

ChipWhisperer Nano Device
io = 
    tio1   = None
    tio2   = None
    tio3   = None
    tio4   = None
    pdid   = True
    pdic   = False
    nrst   = True
    clkout = 7500000.0
adc = 
    clk_src  = int
    clk_freq = 7500000.0
    samples  = 5000
glitch = 
    repeat     = 6
    ext_offset = 546



### Glitching a Single Point

Unlike with the previous tutorial, we don't control when the device sends serial data back to us. This means we'll need to parse the data we get back.

We start our attack off by flushing the ChipWhipserer's serial buffer:

```Python
target.ser.flush()
```

Next, we'll set our trigger source to be "ext_continuous". This differs from "ext_single" in that the ChipWhisperer doesn't need to be armed to insert a glitch, making our loop a little simpler:

```Python
scope.glitch.trigger_src = "ext_continuous"
```

A key part of parsing the serial data is to be able to read a line of data (data terminated with "\n"). We can do that by reading back data until we get a newline character ("\n"):

```Python
while "\n" not in line:
    time.sleep(0.1)
    num_char = target.ser.inWaiting()
    line += target.ser.read(num_char)
```

This needs to be repeated twice in our loop: once at the start to make sure we're on a newline (so we don't look at the wrong numbers for our glitch) and again to actually read the line. For the first read, we also need to make sure we keep any characters after the newline, as this will be the start of the actual line we parse. All together, this looks like:

```Python
line = ""
while "\n" not in line:
    time.sleep(0.1)
    num_char = target.ser.inWaiting()
    line += target.ser.read(num_char)
lines = line.split("\n") 
if len(lines) > 1:
    line = lines[-1]
else:
    line = ""

while "\n" not in line:
    time.sleep(0.1)
    num_char = target.ser.inWaiting()
    line += target.ser.read(num_char)
```

Now that we have our line of data we can parse it by splitting it up via spaces to get each number. 

After the loop ends, we'll need to set our trigger back to "ext_single" to stop the glitches from continuing.

All together (with some additional error checking), this looks like:

In [162]:
from tqdm import tnrange
reset_target(scope)
target.ser.flush()

if SCOPETYPE == "OPENADC":
    scope.glitch.trigger_src = "ext_continuous"

for j in tnrange(20):
    line = ""
    scope.arm()
    while "\n" not in line:
        time.sleep(0.1)
        num_char = target.ser.inWaiting()
        line += target.ser.read(num_char)
    lines = line.split("\n") 
    if len(lines) > 1:
        line = lines[-1]
    else:
        line = ""
    
    while "\n" not in line:
        time.sleep(0.1)
        num_char = target.ser.inWaiting()
        line += target.ser.read(num_char)
        
    if "hello" in line:
        print("Target crashed")
    nums = line.split(" ")
    try:
        if int(nums[0]) != 40000:
            print(line)
        print(lines)
        print(line)
    except ValueError as e:
        continue

if SCOPETYPE == "OPENADC":
    scope.glitch.trigger_src = "ext_single"

['40000 200 200 0', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

Target crashed
['40000 200 200 1', '']
40000 200 200 2
hello

Target crashed
['40000 200 200 0', '400']
40000 200 200 1
hello

Target crashed
['40000 200 200 0', '']
40000 200 200 1
hello

['40000 200 200 0', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200 200 1', 'hello', '']
40000 200 200 0

['40000 200

You should hopefully see some glitched lines printed after running the above block. If not, don't worry -- we'll be adjusting the settings of the glitch which should make them more likely.

### Improving Glitch Settings

One more thing we can do to improve our glitch success rate is to modify our glitch settings (width and offset) while holding the ext offset constant. 

Similar to [Fault_1](Fault_1-Introduction_to_Clock_Glitch_Attacks.ipynb), let's also measure our success rate with each glitch setting, as well as our crash rate, then print them at the end.

If you found an ext offset that worked well for you, be sure to fill it in below:

In [94]:
from tqdm import tnrange, tqdm_notebook


scope.glitch.ext_offset = 2186

Range = namedtuple('Range', ['min', 'max', 'step'])
"""width_range = Range(23, 27, 0.5)
offset_range = Range(-40, 40, 0.5)"""

width_range = Range(38.5, 39.1, 0.4)
offset_range = Range(-28.4, -28.125, 0.4)
scope.glitch.offset_fine = 24
scope.glitch.repeat = 1

reset_target(scope)
target.ser.flush()
scope.glitch.trigger_src = "ext_continuous"

glitches = []
scope.glitch.offset = offset_range.min

t_offset = tqdm_notebook(total=int((offset_range.max-offset_range.min)/offset_range.step) + 1, desc="Offset")
glitch_text = []

if SCOPETYPE == "OPENADC":
    while scope.glitch.offset < offset_range.max:
        scope.glitch.width = width_range.min
        t_width = tqdm_notebook(total=int((width_range.max-width_range.min)/width_range.step), leave=False, desc="Width")
        while scope.glitch.width < width_range.max:
            successes = 0
            crashes = 0
            for j in tnrange(sample_size, leave=False, desc="Attempt"):
                line = ""
                while "\n" not in line:
                    time.sleep(0.1)
                    num_char = target.ser.inWaiting()
                    if num_char == 0:
                        glitch_off(scope)
                        time.sleep(0.01)
                        glitch_on(scope)
                        break
                    line += target.ser.read(num_char)
                lines = line.split("\n") 
                if len(lines) > 1:
                    line = lines[-1]
                else:
                    line = ""

                while "\n" not in line:
                    time.sleep(0.1)
                    num_char = target.ser.inWaiting()
                    if num_char == 0:
                        scope.io.glitch_off(scope)
                        time.sleep(0.01)
                        scope.io.glitch_on(scope)
                        break
                    line += target.ser.read(num_char)

                nums = line.split(" ")
                if "hello" in line:
                    crashes += 1
                    #print("Target crashed")
                #print(line)
                try:
                    if nums[0] == "":
                        continue
                    if int(nums[0]) != 40000:
                        glitch_text += line
                        successes += 1
                except ValueError as e:
                    continue
            glitches.append([scope.glitch.width, scope.glitch.offset, successes / sample_size, crashes / sample_size])
            if successes > 0:
                print([scope.glitch.width, scope.glitch.offset, successes / sample_size, crashes / sample_size])
            scope.glitch.width += width_range.step
            t_width.update()

        scope.glitch.offset += offset_range.step
        t_offset.update()
        t_width.close()
elif SCOPETYPE == "CWNANO":
    

t_width.close()
t_offset.close()
scope.glitch.trigger_src = "ext_single"

[38.671875, -28.515625, 0.628, 0.0]


[39.0625, -28.515625, 0.762, 0.146]


Then, sorting by success rate:

In [89]:
def sort_glitch(glitch):
    return glitch[2]

glitches.sort(key=sort_glitch,reverse=True)
for glitch in glitches:
    print(glitch)

[38.671875, -28.515625, 0.79, 0.0]
[39.0625, -28.515625, 0.76, 0.15]


### Going Further

There's a lot more you can do with this attack:
* If you still weren't able to get any glitches, create an attack loop that scans a larger range of offsets and ranges. You may also have to scan the offset_fine and width_fine glitch settings to find glitches.
* Glitching different instructions will produce different results. Try using your best glitch settings from the previous part and scanning ext offset ranges to produce different glitches
    * During our attack loop, we only checked for glitches in the first number (40000). You may want to examine the other numbers for glitches as well
    * Open the listing file (`.lss`) and view the assembly of the `glitch_infinite()` function. Can you explain the different glitch effects you saw?
* Try using the glitch settings you found in this tutorial to glitch other functions in the `glitchsimple.c` file.

Otherwise, we're done with the tutorial. You can now disconnect from the ChipWhisperer:

In [ ]:
scope.dis()
target.dis()

## Conclusion

With the tutorial now finished, you should have some Vcc glitching experience under your belt. If you're interested in doing more Vcc glitching, you may want to try [Tutorial A9](https://wiki.newae.com/Tutorial_A9_Bypassing_LPC1114_Read_Protect) from the ChipWhisperer Wiki, which uses Vcc glitching to bypass code readout protection on an LPC1114 (requires an LPC1114 dev board). You may also want to try glitching some of the other functions in `glitchsimple.c`. If you have a Raspberry Pi, you can also attempt the attack described [here](https://wiki.newae.com/Tutorial_A3_VCC_Glitch_Attacks#Glitching_More_Advanced_Targets:_Raspberry_Pi) (though you'll need to transfer the steps from the old GUI over to Jupyter).

## Tests

In [ ]:
# Test we got at least one success
# Should be sorted by success rate, so just check first one
success = glitches[0][2] > 0
assert success, "Failed to glitch target"